In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
from scipy import stats
import os

outdir = "plots"
if not os.path.exists(outdir):
    os.makedirs(outdir)

In [ ]:
def load_and_process_data():
    """Load all CSV files and combine data by device type"""
    
    # Find all CSV files matching the pattern
    csv_files = glob.glob("Responses - MERGE_*.csv")
    
    laptop_data = []
    bridge_data = []
    
    for file in csv_files:
        # Extract device type from filename
        if "Laptop" in file:
            device_type = "Laptop"
        elif "Bridge" in file:
            device_type = "Bridge"
        else:
            continue
            
        # Read CSV file
        df = pd.read_csv(file)

        # Remove all occurrences of "\n" from the first row (header)
        df.columns = df.columns.str.replace("\n", " ", regex=False)
        
        # Remove timestamp column and store data
        numeric_data = df.iloc[:, 1:]  # Skip timestamp column
        
        if device_type == "Laptop":
            laptop_data.append(numeric_data)
        else:
            bridge_data.append(numeric_data)
    
    # Combine data for each device type
    laptop_combined = pd.concat(laptop_data, ignore_index=True) if laptop_data else pd.DataFrame()
    bridge_combined = pd.concat(bridge_data, ignore_index=True) if bridge_data else pd.DataFrame()
    
    return laptop_combined, bridge_combined

laptop_data, bridge_data = load_and_process_data()

In [ ]:


def calculate_stats(data):
    """Calculate mean and standard error for each column"""
    means = data.mean()
    std_errors = data.sem()  # Standard error of the mean
    return means, std_errors

    
if laptop_data.empty or bridge_data.empty:
    assert False, "Error: Could not find or load CSV files. Please check file names and paths."

print(f"Loaded {len(laptop_data)} laptop responses and {len(bridge_data)} bridge responses")

# Calculate statistics
laptop_means, laptop_errors = calculate_stats(laptop_data)
bridge_means, bridge_errors = calculate_stats(bridge_data)

# Print summary statistics
print("\n=== SUMMARY STATISTICS ===")
print("\nLaptop Averages:")
for i, (col, mean) in enumerate(laptop_means.items()):
    print(f"{i+1:2d}. {col[:50]}{'...' if len(col) > 50 else ''}: {mean:.2f} ± {laptop_errors.iloc[i]:.2f}")

print("\nBridge Averages:")
for i, (col, mean) in enumerate(bridge_means.items()):
    print(f"{i+1:2d}. {col[:50]}{'...' if len(col) > 50 else ''}: {mean:.2f} ± {bridge_errors.iloc[i]:.2f}")



In [ ]:
sus_short_labels = {
    'I think that I would like to use this system frequently.' : "Use Frequently",
    'I found the system unnecessarily complex.' : "Unnecessarily Complex",
    'I thought this system was easy to use.' : "Easy to Use",
    'I think that I would need the support of a technical person to be able to use this system.' : "Need Tech Support",
    'I found the various functions in this system were well integrated.' : "Well Integrated Functions",
    'I thought there was too much inconsistency in this system.' : "Too Inconsistent",
    'I would imagine that most people would learn to use this system very quickly.' : "Learn Quickly",
    'I found this system very cumbersome to use.' : "Cumbersome",
    'I felt very confident using this system.' : "Confident Using",
    'I needed to learn a lot of things before I could get going with this system.' : "Learn Many Things",
}

def add_carriage_returns(text, max_length=10):
    """Add carriage returns to long text strings for better readability."""
    words = text.split()
    lines = []
    current_line = ""
    
    for word in words:
        if len(current_line) + len(word) + 1 <= max_length:
            current_line += " " + word if current_line else word
        else:
            lines.append(current_line)
            current_line = word
            
    if current_line:
        lines.append(current_line)
        
    return "\n".join(lines)

In [ ]:

# Create plots
print("\nCreating SUS questionnaire plot...")

"""Create bar plot for SUS questionnaire"""

sus_questions = list(sus_short_labels.keys())

for question in sus_questions:
    assert question.strip() in laptop_means.index, f"Column '{question}' not found in laptop_means"
    assert question.strip() in laptop_errors.index, f"Column '{question}' not found in laptop_errors"
    assert question.strip() in bridge_means.index, f"Column '{question}' not found in bridge_means"
    assert question.strip() in bridge_errors.index, f"Column '{question}' not found in bridge_errors"

# Create laptop_sus, laptop_sus_err, bridge_sus, bridge_sus_err using all columns in sus_questions
laptop_sus = laptop_means[sus_questions]
laptop_sus_err = laptop_errors[sus_questions]
bridge_sus = bridge_means[sus_questions]
bridge_sus_err = bridge_errors[sus_questions]


x = np.arange(len(sus_short_labels))
width = 0.35

fig, ax = plt.subplots(figsize=(8, 4))
ax.yaxis.grid(True, linestyle='--', alpha=0.7, zorder=-1)

bars2 = ax.bar(x - width/2, bridge_sus, width, label='Bridge', edgecolor='black', 
                yerr=bridge_sus_err, capsize=5, zorder = 100)
bars1 = ax.bar(x + width/2, laptop_sus, width, label='Laptop', edgecolor='black', 
                yerr=laptop_sus_err, capsize=5, zorder = 100)

# ax.set_xlabel('SUS Questions')
ax.set_ylabel('Average SUS Score (1-5 scale)')
# ax.set_title('System Usability Scale (SUS) Scores by Device')
ax.set_xticks(x)
ax.set_xticklabels([add_carriage_returns(sus_short_labels[e]) for e in laptop_sus.index], 
                   rotation=35, 
                   ha='center')
ax.legend(loc='upper center')
# Grid on y-axis
ax.set_ylim(0, 5.5)

# Add value labels on bars
# for bar in bars1:
#     height = bar.get_height()
#     ax.annotate(f'{height:.2f}',
#                 xy=(bar.get_x() + bar.get_width() / 2, height),
#                 xytext=(0, 3),  # 3 points vertical offset
#                 textcoords="offset points",
#                 ha='center', va='bottom', fontsize=8)

# for bar in bars2:
#     height = bar.get_height()
#     ax.annotate(f'{height:.2f}',
#                 xy=(bar.get_x() + bar.get_width() / 2, height),
#                 xytext=(0, 3),  # 3 points vertical offset
#                 textcoords="offset points",
#                 ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.savefig(os.path.join(outdir,"sus_plot.png"), dpi=300, bbox_inches='tight')
plt.show()


In [ ]:

print("Creating ad-hoc questions plot...")
"""Create bar plot for ad-hoc questions"""

# Short labels for ad-hoc questions
adhoc_short_labels = {
    'How responsive was the system': "Responsive",
    'How precise was the system': "Precise",
}

adhoc_columns = list(adhoc_short_labels.keys())
for question in adhoc_columns:
    assert question.strip() in laptop_means.index, f"Column '{question}' not found in laptop_means"
    assert question.strip() in laptop_errors.index, f"Column '{question}' not found in laptop_errors"
    assert question.strip() in bridge_means.index, f"Column '{question}' not found in bridge_means"
    assert question.strip() in bridge_errors.index, f"Column '{question}' not found in bridge_errors"
# Use last 2 columns (ad-hoc questions)
laptop_adhoc = laptop_means[adhoc_columns]
laptop_adhoc_err = laptop_errors[adhoc_columns]
bridge_adhoc = bridge_means[adhoc_columns]
bridge_adhoc_err = bridge_errors[adhoc_columns]

x = np.arange(len(adhoc_short_labels))
width = 0.35

fig, ax = plt.subplots(figsize=(4.3, 3))

bars1 = ax.bar(x - width/2, laptop_adhoc, width, label='Laptop', edgecolor='black',
                yerr=laptop_adhoc_err, capsize=5)
bars2 = ax.bar(x + width/2, bridge_adhoc, width, label='Bridge', edgecolor='black', 
                yerr=bridge_adhoc_err, capsize=5)

# ax.set_xlabel('Ad-hoc Questions')
ax.set_ylabel('Average Score (1-7 scale)')
# ax.set_title('Ad-hoc System Evaluation by Device')
ax.set_xticks(x)
ax.set_xticklabels([adhoc_short_labels[e] for e in laptop_adhoc.index], 
                #    rotation=30,
                     ha='center')
ax.legend(loc="upper center")
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 7.5)

# Add value labels on bars
# for bar in bars1:
#     height = bar.get_height()
#     ax.annotate(f'{height:.2f}',
#                 xy=(bar.get_x() + bar.get_width() / 2, height),
#                 xytext=(0, 3),  # 3 points vertical offset
#                 textcoords="offset points",
#                 ha='center', va='bottom', fontsize=10)

# for bar in bars2:
#     height = bar.get_height()
#     ax.annotate(f'{height:.2f}',
#                 xy=(bar.get_x() + bar.get_width() / 2, height),
#                 xytext=(0, 3),  # 3 points vertical offset
#                 textcoords="offset points",
#                 ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig(os.path.join(outdir,"adhoc_plot.png"), dpi=300, bbox_inches='tight')
plt.show()